In [2]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person
from tqdm import tqdm
import json

In [3]:
auth_file = "/home/ivan/pascal_adversarial_faces/azure_auth.json"
with open(auth_file, "r") as f:
    auth_data = json.loads(f.read())


In [4]:
face_client = FaceClient(
    auth_data["endpoint"], 
    CognitiveServicesCredentials(auth_data["key"])
)

In [5]:
# face_client.person_group.delete(person_group_id="robust_community_naive_mean_1_1_0p5")

In [15]:
def measure_azure_recall(
    face_client,
    image_directory,
    azure_person_group_name
):
    
    person_id_to_name = {}
    identities = []
    for p in face_client.person_group_person.list(person_group_id=azure_person_group_name):
        person_id_to_name[p.person_id] = p.name
        identities.append(p.name)

    discovery = []
    true = []
    identified_as = []

    for protector in tqdm(identities):
        # We are sourcing query photos from epsilon_0.0.
        # In those cases, all subfolders in the "protected" identity have the same, clean
        # photo of the protector, so we just pick any single one that exists (e.g. n000958)
        # For the case where n000958 is itself the protector, n000958 is not present in its protected
        # subfolders, so we pick n000029 without loss of generality.
        if protector == "n000958":
            protected = "n000029"
        else:
            protected = "n000958"

        query_photos_paths = sorted(glob.glob(
            f"{image_directory}/{protector}/community_naive_mean/{protected}/epsilon_0.0/png/*"
        ))
        
        # For this person group, we picked the first 10 lexicographically sorted
        # photos to use in the lookup set, so for the query, we use the 11th and on.
        # (The query is not supposed to be in the lookup set).
        for i in range(11, len(query_photos_paths)):
            faces_in_query_photos = face_client.face.detect_with_stream(
                open(query_photos_paths[i], "r+b"), 
                detectionModel='detection_02'
            )
            if len(faces_in_query_photos) > 0:
                break


        # There should only be one face, so we use that as the query face.
        results = face_client.face.identify(
            [faces_in_query_photos[0].face_id],
            azure_person_group_name
        )
        
        true.append(protector)

        if len(results) < 1 or len(results[0].candidates) < 1:
            discovery.append(0.0)
            identified_as.append("None")

        else: 
            top_identity = person_id_to_name[results[0].candidates[0].person_id]

            identified_as.append(top_identity)
            
            # Note the switch of the term protector here:
            # protectors are also protected but we call them protectors because of the folder structure
            # In this case, the query photo belongs to the protector -- who is also protected by decoys 
            # of *other* protectors. Therefore, if the identity returned is that of the "protector,"
            # this is a failure in the defense.
            if top_identity == protector:
                discovery.append(1.0)
            else:
                discovery.append(0.0)

        time.sleep(10)
    
    for true_id, recognized_id in zip(true, identified_as):
        print(f"Face of {true_id} identitifed as {recognized_id}")
        
    return sum(discovery)/len(discovery)

In [25]:
measure_azure_recall(
    face_client,
    "/data/vggface/test_perturbed_sampled",
    "robust_community_naive_mean_10_5_0p5"
)



  0%|          | 0/19 [00:00<?, ?it/s]

  5%|▌         | 1/19 [00:10<03:07, 10.43s/it]

 11%|█         | 2/19 [00:20<02:56, 10.37s/it]

 16%|█▌        | 3/19 [00:30<02:45, 10.34s/it]

 21%|██        | 4/19 [00:41<02:34, 10.32s/it]

 26%|██▋       | 5/19 [00:51<02:23, 10.28s/it]

 32%|███▏      | 6/19 [01:01<02:13, 10.28s/it]

 37%|███▋      | 7/19 [01:12<02:04, 10.35s/it]

 42%|████▏     | 8/19 [01:22<01:53, 10.31s/it]

 47%|████▋     | 9/19 [01:32<01:42, 10.30s/it]

 53%|█████▎    | 10/19 [01:42<01:32, 10.28s/it]

 58%|█████▊    | 11/19 [01:53<01:22, 10.27s/it]

 63%|██████▎   | 12/19 [02:03<01:11, 10.27s/it]

 68%|██████▊   | 13/19 [02:13<01:01, 10.25s/it]

 74%|███████▎  | 14/19 [02:23<00:51, 10.27s/it]

 79%|███████▉  | 15/19 [02:34<00:41, 10.30s/it]

 84%|████████▍ | 16/19 [02:44<00:30, 10.31s/it]

 89%|████████▉ | 17/19 [02:54<00:20, 10.29s/it]

 95%|█████████▍| 18/19 [03:05<00:10, 10.35s/it]

100%|██████████| 19/19 [03:15<00:00, 10.30s/it]

Face of n002763 identitifed as n002763
Face of n007548 identitifed as n007548
Face of n004658 identitifed as n004658
Face of n002503 identitifed as None
Face of n009232 identitifed as n009232
Face of n002647 identitifed as n002647
Face of n000958 identitifed as n000958
Face of n003356 identitifed as n003356
Face of n001683 identitifed as n001683
Face of n009288 identitifed as None
Face of n008613 identitifed as n008613
Face of n005303 identitifed as n005303
Face of n008655 identitifed as None
Face of n001781 identitifed as n001781
Face of n005359 identitifed as None
Face of n003215 identitifed as None
Face of n000029 identitifed as n000029
Face of n009114 identitifed as None
Face of n005427 identitifed as n005427


0.6842105263157895

In [16]:
measure_azure_recall(
    face_client,
    "/data/vggface/test_perturbed_sampled",
    "robust_community_naive_mean_1_1_0p5"
)



  0%|          | 0/19 [00:00<?, ?it/s]

  5%|▌         | 1/19 [00:10<03:04, 10.24s/it]

 11%|█         | 2/19 [00:20<02:53, 10.23s/it]

 16%|█▌        | 3/19 [00:30<02:43, 10.22s/it]

 21%|██        | 4/19 [00:40<02:33, 10.21s/it]

 26%|██▋       | 5/19 [00:51<02:23, 10.21s/it]

 32%|███▏      | 6/19 [01:01<02:13, 10.24s/it]

 37%|███▋      | 7/19 [01:11<02:03, 10.26s/it]

 42%|████▏     | 8/19 [01:21<01:52, 10.25s/it]

 47%|████▋     | 9/19 [01:32<01:42, 10.26s/it]

 53%|█████▎    | 10/19 [01:42<01:32, 10.24s/it]

 58%|█████▊    | 11/19 [01:52<01:21, 10.24s/it]

 63%|██████▎   | 12/19 [02:02<01:11, 10.24s/it]

 68%|██████▊   | 13/19 [02:13<01:01, 10.25s/it]

 74%|███████▎  | 14/19 [02:23<00:51, 10.26s/it]

 79%|███████▉  | 15/19 [02:33<00:41, 10.26s/it]

 84%|████████▍ | 16/19 [02:43<00:30, 10.26s/it]

 89%|████████▉ | 17/19 [02:54<00:20, 10.26s/it]

 95%|█████████▍| 18/19 [03:04<00:10, 10.25s/it]

100%|██████████| 19/19 [03:14<00:00, 10.24s/it]

Face of n000029 identitifed as n000029
Face of n002763 identitifed as n002763
Face of n000958 identitifed as n000958
Face of n005359 identitifed as n005359
Face of n004658 identitifed as n004658
Face of n009232 identitifed as n009232
Face of n005303 identitifed as n005303
Face of n001683 identitifed as n001683
Face of n009288 identitifed as n009288
Face of n007548 identitifed as n007548
Face of n008613 identitifed as n008613
Face of n008655 identitifed as None
Face of n003356 identitifed as n003356
Face of n001781 identitifed as n001781
Face of n003215 identitifed as n003215
Face of n009114 identitifed as n009114
Face of n002503 identitifed as n002503
Face of n002647 identitifed as n002647
Face of n005427 identitifed as n005427


0.9473684210526315